In [54]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from esr21_metadata_rules.predicates import SubjectPredicates

pc = SubjectPredicates()

site_ids = (40, 41, 42, 43, 44,)



## AE Rules
#### Missing Serious AE

In [55]:

pids_with_missing_sae = list()


ae_crfs = AdverseEvent.objects.filter(site_id__in=site_ids)

for crf in ae_crfs:
    
    sae_predicate = pc.func_serious_ae_required(crf.subject_visit)

    if sae_predicate:

        sae_exists = SeriousAdverseEvent.objects.filter(subject_visit=crf.subject_visit).exists()
        
        if not sae_exists:
            pids_with_missing_sae.append(crf.subject_visit)



In [57]:
pids_with_missing_sae

[<SubjectVisit: 150-040993693-5 1014.0>,
 <SubjectVisit: 150-040993564-8 1098.1>]

## AE Rules
#### Missing Special AE 

In [58]:

pids_with_missing_ssae = list()

ae_crfs = AdverseEvent.objects.filter(site_id__in=site_ids)

for crf in ae_crfs:
    

    sae_predicate = pc.func_special_interest_ae_required(crf.subject_visit)

    if sae_predicate:

        sae_exists = SpecialInterestAdverseEvent.objects.filter(subject_visit=crf.subject_visit)

        if not sae_exists:
            pids_with_missing_ssae.append(crf)



In [59]:
pids_with_missing_ssae

[<AdverseEvent: 150-040991578-0 1007.0>,
 <AdverseEvent: 150-041971348-0 1098.1>]

## Covid-19 Rules
#### Missing physical exam and vaccination details

In [64]:
from edc_constants.choices import NEG


covid_results_crfs = Covid19Results.objects.filter(covid_result=NEG, site_id__in=site_ids)

missing_physical_exam = list()
missing_vaccination_details = list()

for crf in covid_results_crfs:
    
    physical_exam_exist = PhysicalExam.objects.filter(subject_visit=crf.subject_visit).exists()
    vaccination_details_exist = VaccinationDetails.objects.filter(subject_visit=crf.subject_visit).exists()

    if not physical_exam_exist:
        missing_physical_exam.append(crf)

    if not vaccination_details_exist:
        missing_vaccination_details.append(crf)
            
                


In [63]:
missing_physical_exam

[<Covid19Results: 150-040992893-2 1077.0>,
 <Covid19Results: 150-040990805-8 1084.0>,
 <Covid19Results: 150-041970741-7 1007.0>,
 <Covid19Results: 150-040992189-5 1014.0>,
 <Covid19Results: 150-040992968-2 1077.0>,
 <Covid19Results: 150-040993649-7 1070.0>,
 <Covid19Results: 150-040990990-8 1084.0>,
 <Covid19Results: 150-040993649-7 1070.1>,
 <Covid19Results: 150-040991126-8 1007.0>,
 <Covid19Results: 150-041971712-7 1007.0>,
 <Covid19Results: 150-040991409-8 1007.0>,
 <Covid19Results: 150-040990323-2 1014.0>,
 <Covid19Results: 150-040990376-0 1098.0>,
 <Covid19Results: 150-040993438-5 1014.0>,
 <Covid19Results: 150-040991409-8 1077.0>,
 <Covid19Results: 150-040990157-4 1007.0>,
 <Covid19Results: 150-040992839-5 1007.0>,
 <Covid19Results: 150-040993564-8 1007.0>,
 <Covid19Results: 150-041970528-8 1000.0>,
 <Covid19Results: 150-040992404-8 1014.0>,
 <Covid19Results: 150-041971575-8 1028.0>,
 <Covid19Results: 150-040990164-0 1182.0>,
 <Covid19Results: 150-040990041-0 1014.0>,
 <Covid19Re

In [65]:
missing_vaccination_details

[<Covid19Results: 150-040992893-2 1077.0>,
 <Covid19Results: 150-040990805-8 1084.0>,
 <Covid19Results: 150-041970741-7 1007.0>,
 <Covid19Results: 150-040992189-5 1014.0>,
 <Covid19Results: 150-040992968-2 1077.0>,
 <Covid19Results: 150-040993649-7 1070.0>,
 <Covid19Results: 150-040990990-8 1084.0>,
 <Covid19Results: 150-040993649-7 1070.1>,
 <Covid19Results: 150-040991126-8 1007.0>,
 <Covid19Results: 150-041971712-7 1007.0>,
 <Covid19Results: 150-040991409-8 1007.0>,
 <Covid19Results: 150-040990323-2 1014.0>,
 <Covid19Results: 150-040990376-0 1098.0>,
 <Covid19Results: 150-040993438-5 1014.0>,
 <Covid19Results: 150-040991409-8 1077.0>,
 <Covid19Results: 150-040990157-4 1007.0>,
 <Covid19Results: 150-040992839-5 1007.0>,
 <Covid19Results: 150-040993564-8 1007.0>,
 <Covid19Results: 150-041970528-8 1000.0>,
 <Covid19Results: 150-040992404-8 1014.0>,
 <Covid19Results: 150-041971575-8 1028.0>,
 <Covid19Results: 150-040990164-0 1182.0>,
 <Covid19Results: 150-040990041-0 1014.0>,
 <Covid19Re

## Medical History Rules

#### Missing Concomitant medication

In [ ]:



missing_concomitant_medication = list()

medical_histories = MedicalHistory.objects.filter(site_id__in=site_ids)

for mh in medical_histories:
    
    concomitant_predicate = pc.fun_conc_med_required(mh.subject_visit)
    
    if concomitant_predicate:
        
        concomitant_exist = ConcomitantMedication.objects.filter(subject_visit=mh.subject_visit)
        
        if not concomitant_exist:
            
            missing_concomitant_medication.append(mh)
    
    


In [71]:
missing_concomitant_medication

[<MedicalHistory: 150-043940128-0 1000.0>,
 <MedicalHistory: 150-043941507-4 1170.0>,
 <MedicalHistory: 150-042980988-0 1000.0>,
 <MedicalHistory: 150-043940317-9 1000.0>,
 <MedicalHistory: 150-041970141-0 1000.0>,
 <MedicalHistory: 150-044960198-6 1000.0>,
 <MedicalHistory: 150-041971869-5 1170.0>,
 <MedicalHistory: 150-041970753-2 1000.0>,
 <MedicalHistory: 150-041971877-8 1170.0>,
 <MedicalHistory: 150-041970070-1 1000.0>,
 <MedicalHistory: 150-042981283-5 1170.0>,
 <MedicalHistory: 150-042981059-9 1000.0>,
 <MedicalHistory: 150-041970082-6 1000.0>,
 <MedicalHistory: 150-044960626-6 1000.0>,
 <MedicalHistory: 150-040990892-6 1000.0>,
 <MedicalHistory: 150-040991111-0 1000.0>,
 <MedicalHistory: 150-042980019-4 1000.0>,
 <MedicalHistory: 150-040993019-3 1000.0>,
 <MedicalHistory: 150-042980074-9 1000.0>,
 <MedicalHistory: 150-044960725-6 1000.0>,
 <MedicalHistory: 150-041971865-3 1170.0>,
 <MedicalHistory: 150-042980549-0 1000.0>,
 <MedicalHistory: 150-041970869-6 1000.0>,
 <MedicalHi

In [ ]:
len(missing_concomitant_medication)

## Pregnancy Status Rules
#### Missing preg. test

In [ ]:
missing_pregnancy_tests = list()

pregnancy_statuses = PregnancyStatus.objects.filter(site_id__in=site_ids)

for pregnancy_status in pregnancy_statuses:
    
    pregnancy_test_predicate = pc.func_preg_test_required(subject_visit=pregnancy_status.subject_visit)
    
    if pregnancy_test_predicate:
        
        pregnancy_test_exists = PregnancyTest.objects.filter(subject_visit=pregnancy_status.subject_visit).exists()
        
        if not pregnancy_test_exists:
            missing_pregnancy_tests.append(pregnancy_status)
    

In [ ]:
missing_pregnancy_tests

## Vaccination Details Rule
#### Missing AE

In [74]:
from edc_constants.constants import YES

vaccination_details = VaccinationDetails.objects.filter(adverse_event = YES)

missing_ae = []

for crf in vaccination_details:
    
    adverse_event_exist = AdverseEvent.objects.filter(subject_visit=crf.subject_visit)
    
    if not adverse_event_exist:
        missing_ae.append(crf)
        


EncryptionError. Got Incorrect decryption. Got b"Q\xd2\xb4\x04L\x1f\x98\xe2\xac\xfaa/5\xda\xce\x19V\xc3\x8b>\xd0L\x9d\x14f~\x84\x8a@\xde\xd5$\xc6\xben\xe2\x19\xcde\xea\x07xT\x12\x81\xb4\xa2g\xab\x14\x833\x99\xec\xf5\xa4\\\xae\xf8\xcd\x0c\xe2\x1e8\xe8\x00\x12n]\x84\x91\xc2#n\xed\x02\x7fG\xeeSU]3\xa9 \xb4\xf7\xa9\xbc\xd43\x7f\x98r\x12M\x93\x9b\xc61\x8eU\xd1og\xd8M\xac\n\x1fa`|\x99\xae R\xa9L$\xbe\xe1\xce^h\xef\xb1\xe7\xa3\x18u\xf2\x9c\xdb,^f\x94<\x93\xb1[~\x1a\xde\xd7\xa8\r\x19g\xcaO\x01\xd3S\xc7\\\x1a\xa5w\x12H\xa0\x18\xe8\x86\x1d\xc7\x90\xda\xefO\xd8\\!Ny{\x1d\x1b\x1bP@1\xc5\x02{\x1d\xee\xa4X@\xb3\xa4\xb1\xb5(FEj\xfb\x1eY'#\x97\x9e\xb4\xd2\xdf \x9a\xff\x0f\xef:\xb8\xb5_v'\xac\x99\xc2hdE\xfbL\xfe\x10\xfe;\x11j\xc2.>\x05\xfa\xd6>\x0c\x17q\xa8\x98\xc9\xb7FO\x03\xa49\x1f\xbe".


EncryptionError: Incorrect decryption. Got b"Q\xd2\xb4\x04L\x1f\x98\xe2\xac\xfaa/5\xda\xce\x19V\xc3\x8b>\xd0L\x9d\x14f~\x84\x8a@\xde\xd5$\xc6\xben\xe2\x19\xcde\xea\x07xT\x12\x81\xb4\xa2g\xab\x14\x833\x99\xec\xf5\xa4\\\xae\xf8\xcd\x0c\xe2\x1e8\xe8\x00\x12n]\x84\x91\xc2#n\xed\x02\x7fG\xeeSU]3\xa9 \xb4\xf7\xa9\xbc\xd43\x7f\x98r\x12M\x93\x9b\xc61\x8eU\xd1og\xd8M\xac\n\x1fa`|\x99\xae R\xa9L$\xbe\xe1\xce^h\xef\xb1\xe7\xa3\x18u\xf2\x9c\xdb,^f\x94<\x93\xb1[~\x1a\xde\xd7\xa8\r\x19g\xcaO\x01\xd3S\xc7\\\x1a\xa5w\x12H\xa0\x18\xe8\x86\x1d\xc7\x90\xda\xefO\xd8\\!Ny{\x1d\x1b\x1bP@1\xc5\x02{\x1d\xee\xa4X@\xb3\xa4\xb1\xb5(FEj\xfb\x1eY'#\x97\x9e\xb4\xd2\xdf \x9a\xff\x0f\xef:\xb8\xb5_v'\xac\x99\xc2hdE\xfbL\xfe\x10\xfe;\x11j\xc2.>\x05\xfa\xd6>\x0c\x17q\xa8\x98\xc9\xb7FO\x03\xa49\x1f\xbe".

In [ ]:
missing_ae

## Subject Visit Rule
#### Missing preg. status, vaccination details, physical details, preg. outcome, demographicdata, rapid hiv testing and covid prev and medical history

In [75]:

subject_visits = SubjectVisit.objects.filter(site_id__in=site_ids)

missing_pregnancy = []
missing_covid_symptoms = []
missing_preg_outcome = []
missing_new_participants = []

for visit in subject_visits:
    
    pregnancy_predicate = pc.func_participant_female(visit)
    
    covid_symptoms_predicate = pc.func_symptomatic_infection_enrol(visit)
    
    preg_outcome_predicate = pc.func_preg_outcome_required(visit)
    
    new_participants_predicate = pc.fun_enrol_forms_required(visit)
    
    if pregnancy_predicate:
        preg_exists = PregnancyStatus.objects.filter(subject_visit=visit).exists()
        if not preg_exists:
            missing_pregnancy.append(subject_visits)
            
    if covid_symptoms_predicate:
        phyisical_exam_exists = PhysicalExam.objects.filter(subject_visit=visit).exists()
        vaccination_details_exist = VaccinationDetails.objects.filter(subject_visit=visit).exists()
        
        if not phyisical_exam_exists:
            missing_covid_symptoms.append(f'{subject_visits} - physicalexam')
            
        if not vaccination_details_exist:
            missing_covid_symptoms.append(f'{subject_visits} - vaccinationdetails')
            
    if preg_outcome_predicate:
        preg_exists = PregOutcome.objects.filter(subject_visit=visit).exists()
        if not preg_exists:
            missing_preg_outcome.append(subject_visits)
            
            
    if new_participants_predicate:
        demo_exists = DemographicsData.objects.filter(subject_visit=visit).exists()
        rapid_hiv_exists = RapidHIVTesting.objects.filter(subject_visit=visit).exists()
        covid_b_exists = Covid19PreventativeBehaviours.objects.filter(subject_visit=visit).exists()
        medicah_hx_exists = MedicalHistory.objects.filter(subject_visit=visit).exists()
        
        if not demo_exists:
            missing_pregnancy.append(f'{subject_visits} - demographicsdata')
            
        if not rapid_hiv_exists:
            missing_pregnancy.append(f'{subject_visits} - rapidhivtesting')
            
        if not covid_b_exists:
            missing_pregnancy.append(f'{subject_visits} - covid19preventativebehaviours')
            
        if not medicah_hx_exists:
            missing_pregnancy.append(f'{subject_visits} - medicalhistory')
            
    
    



EncryptionError. Got Incorrect decryption. Got b"5\x92\xf7<?n\x0c\x87\x1d\xa8ck\xe4\x18\xd3g\xd2\xe09\xb7z\xbb\x92G\xd5\xff/\xa35Dq\xe9/Nv\xa7,h\x9b\x8df\x82\x9b A$\x1b\xef\x8a8K\x83:a\xaf\x8cO\x04\x05\xcci\x99\\1\x9b_\xd6\x9a\x98/\x8d\xf6\x8bV\x94\xd2\x87\xd3\xbbl\x9b1K\x91\xf4\xd6A\x13TDU\xcd\x16\xab4\xe3\x12\xb3\xeb\x16\xeb\x0eh\x08\xd3\xee\x9e\xe9C\xc2\xbb\xbeM\xe45S_\xc3\x17\xdb~\x84\xf3\x07\x90\xa3\xf5c\xec\xcd\x8ci\x8f0\xf4\xb8C\xdd\xff\xae\x936E0\xcc\xc3L\xf7u7ii\xcf\xd6\xed\x82o\x8d\xbcFLeD\x1a>\\h\x0f\xd6\xb7\xda\x17\x08\xb4\x94|<\xa5\xb1\x1c\x1e\x1e'PA \xe1\xfa\xe8X\xdbm\xf6\x99\xf3\xf0\x87\x0b\xb9\xf6E\xc7\xf7G\xa4\x17\xdf\x1b\x04a\x9c]\x14~\x8c\xf5\x0b7\xf5\xb3\xda\x0e\xcf\x84\xe6\x99\x96c\xba~3\x03\xf0\x86\xd4\xaf\x89\x01\xde\xb3\xb8l\xd3\xae\x10\xdbC1S5N\xaf=R\xc1v".


EncryptionError: Incorrect decryption. Got b"5\x92\xf7<?n\x0c\x87\x1d\xa8ck\xe4\x18\xd3g\xd2\xe09\xb7z\xbb\x92G\xd5\xff/\xa35Dq\xe9/Nv\xa7,h\x9b\x8df\x82\x9b A$\x1b\xef\x8a8K\x83:a\xaf\x8cO\x04\x05\xcci\x99\\1\x9b_\xd6\x9a\x98/\x8d\xf6\x8bV\x94\xd2\x87\xd3\xbbl\x9b1K\x91\xf4\xd6A\x13TDU\xcd\x16\xab4\xe3\x12\xb3\xeb\x16\xeb\x0eh\x08\xd3\xee\x9e\xe9C\xc2\xbb\xbeM\xe45S_\xc3\x17\xdb~\x84\xf3\x07\x90\xa3\xf5c\xec\xcd\x8ci\x8f0\xf4\xb8C\xdd\xff\xae\x936E0\xcc\xc3L\xf7u7ii\xcf\xd6\xed\x82o\x8d\xbcFLeD\x1a>\\h\x0f\xd6\xb7\xda\x17\x08\xb4\x94|<\xa5\xb1\x1c\x1e\x1e'PA \xe1\xfa\xe8X\xdbm\xf6\x99\xf3\xf0\x87\x0b\xb9\xf6E\xc7\xf7G\xa4\x17\xdf\x1b\x04a\x9c]\x14~\x8c\xf5\x0b7\xf5\xb3\xda\x0e\xcf\x84\xe6\x99\x96c\xba~3\x03\xf0\x86\xd4\xaf\x89\x01\xde\xb3\xb8l\xd3\xae\x10\xdbC1S5N\xaf=R\xc1v".